In [1]:
NO_EDGE = [0][0]

def _find_path(from_vertex, to_vertex, graph, path):
    """Finds a path between two vertices in a graph.

    Inputs
    ------
    source : int
      label of source vertex
    target : int
      label of target vertex
    graph : list of int
      adjacency matrix representation fo graph to explore
    path : list of int
      existing path to source vertex along the way to target

    Returns
    -------
    path : list of int
      The vertices in the path between from_vertex and to_vertex in the order
      they are traversed.

    The method creates a path by finding the next vertex along a possible path.
    If a path exists, eventually we reach the to_vertex and the recursion ends.
    """

    # Capture the no-edge value in the adjacency matrix
    NO_EDGE = graph[0][0]
    # Add the present starting point to the path
    path.append(from_vertex)

    # Check for base case, if we have already reached the target vertex
    if from_vertex == to_vertex:
        return path

    # Find the neighbors of the current from_vertex and, if they are not already
    # in the path, consider them for exploration next.
    for next_vertex in range(len(graph)):
        if graph[from_vertex][next_vertex] != NO_EDGE:
            if next_vertex not in path:
                # Recursive call
                possible_path = _find_path(next_vertex, to_vertex, graph, path)
                if possible_path:
                    return path
    # Sometimes we may go down a road that doesn't lead to the to_vertex and
    # conclude that there is no path between from/to vertices
    return None

def find_path(source, target, graph):
    """User-facing method to start the recursive search for a path. Passes the
    source vertex as the from_vertex for the recursive function, target vertex
    becomes to_vertex, and an empty path list is provided. Method returns None
    is there is no path between source and vertex; otherwise it retuns the a
    path between them.
    """
    path = []
    return _find_path(source, target, graph, path)

def minimum_capacity_edge(path, graph):
    """Finds the minimum capacity of an edge along a given path in a graph."""
    # Assume that weakest edge in path is the first edge
    min_capacity = graph[path[0]][path[1]]
    # Traverse the remaining of the path to find if there is a smaller edge
    for i in range(1,len(path)-1):
        current_capacity = graph[path[i]][path[i+1]]
        if current_capacity < min_capacity:
            min_capacity = current_capacity
    return min_capacity

def saturate(graph, path, flow):
    """Sends a flow down a path. When the flow matches the path's weakest edge
    capacity, the path is saturated. For edges that have capacity avaibable
    after the flow added, a backedge is added to indicate the residual capacity.
    """
    for i in range(len(path)-1):
        # Update the edge (u,v) in the residual graph to show that a flow has
        # been added to the path. Adding flow to an edge reduces its capacity
        # by the same amount
        graph[path[i]][path[i+1]] -= flow
        # Add backflow
        graph[path[i+1]][path[i]] += flow

def find_path_iter(source, target, graph):
    """Finds and returns a path from source to target vertices in a dag given as
    as adjacency matrix graph. If no path is found, method returns None. If a
    path is found, it is returned as a list of vertices in the order they appear
    in the path, e.g.,
      path = [source, next vertex, next, etc..., target]
    """
    NO_EDGE = graph[0][0]
    # Stack is a list of pairs. Each pair is a vertex and a list of vertices
    # that show the path to that vertex
    stack = [(source, [source])]
    # Iterate the stack while it is not empty
    while len(stack) > 0:
        # Pop the stack and obtain the vertex currently in the stack and the
        # path leading to it
        u, path = stack.pop()
        if u == target:
            return path
        # Otherwise consider every neighbor of the current vertex u:
        for v in range(len(graph)):
            if graph[u][v] != NO_EDGE:
                if v not in path:
                    stack.append((v, path+[v]))
    return None


#The minimum cut is the smallest possible set of edges that, when removed from
#the graph, halt any traffic between source and target vertices.

"""
Given a dag G=(V,E) representing a flow network, with two of its vertices
designated as source and target, find the maximum flow of the graph.
The maximum flow of the graph is the sum of the maximum flows of all augmenting
paths in the residual graph.

Also find the graph's minumum cut. This can be done in the same method that
finds the maximum flow.

After all augmenting paths have been saturated, the vertices of the residual
graph are separated into two sets: set S with the vertices that can be
from the source vertex and set T with those that cannot be reached. These sets
are a partition of V, i.e., S∩T=∅ and S∪T=V. The minimum cut C of the graph is
the set:

C={(u,v)∈E | u∈S,v∈T}

ie, all the edges in the input graph whose vertices are not both in S or in T.
"""

#find the maximum flow of the graph
"""
find the maximum flows of all augmenting paths and add them together
1. find paths by calling find_path
2. find max by calling minimum capacity edge

initialize residual graph R
initialize max flow ←0
while there is an augmenting path in R
  max flow ← max flow + capacity of augmenting path
  saturate the augmented path
return max flow

max flow is the same as the min cut capacity, so find the least number
of edge weights that equal max flow

minimum cut has one vertex in u and one vertex in v

"""
def find_max_flow(source, target, graph):
  #create copy
  R = deepcopy(graph)
  #initialize max_flow and path
  max_flow = 0
  path = find_path(source, target, R)
  #initialize min_cut
  min_cut = []
  #while there is a path
  while path:
    #find min_capacity of the path
    min_capacity = minimum_capacity_edge(path, R)
    #update max_flow by adding the capacity
    max_flow = max_flow + min_capacity
    #saturate path
    saturate(R, path, min_capacity)
    #initialize path again to prevent inifinite loop
    path = find_path(source, target, R)
  #find min_cut
  #to find min cut, check each edge and if u and v are not both in S or T
  #add that edge to the min_cut array
  #find all the edges in the graph
  edges = find_edges(graph)
  #find the vertices reachable from source in the residual graph
  S = reachable_from(source, R)
  #find vertices not in s and put them in set T
  T = [v for v in range(len(graph)) if v not in S]
  #for every element in S and for every element in T if graph[u][v] is not empty
  #that means that u and v are in different sets so appened the edge to min_cut
  for u in S:
    for v in T:
      if graph[u][v] != NO_EDGE:
        min_cut.append((u,v))
  return max_flow, min_cut

#method to find reachable and unreachable nodes in G from source
def reachable_from(source, G):
  visited = []
  stack = [source]
  while stack:
    u = stack.pop()
    if u not in visited:
      visited.append(u)
      for v in range(len(G)):
        if G[u][v] != NO_EDGE:
          stack.append(v)
  return visited

#method that finds and returns all the edges in a graph as a list
def find_edges(G):
  edges = []
  for u in range(len(G)):
    for v in range(len(G)):
      if G[u][v] != NO_EDGE:
        edges.append(G[u][v])
  return edges

# Test graph

G = [#  A   B   C   D   E   F
     [  0, 20,  0,  0,  0,  0], # A
     [  0,  0,  5,  6,  0, 10], # B
     [  0,  0,  0,  3,  7,  0], # C
     [  0,  0,  0,  0,  8,  0], # D
     [  0,  0,  0,  0,  0,  0], # E
     [  0,  0,  0,  0,  0,  0]  # F
]

# Dependecies to use
from copy import deepcopy

max_flow_result, min_cut_result = find_max_flow(0, 4, G)
print(max_flow_result)
print(min_cut_result)

11
[(1, 2), (1, 3)]


Assignment 9 Ungrading:

Although my code could be cleaner, it does a decent job at utilizing a min heap. A good chouce I made was to test my code, and even though my results were not 100% accurate, my testing showed me that my thought process was on the right track. I also included comments to explain my thought processes. I did change heapify_up to reflect a min heap, but I should've made more changes to heapify down to reflect a min heap. I also should've made a seperate class for the min_heap instead of create a min heap using the max heap method. While my attempt to utilize a min heap in huffman is a little messy and could be better, it was an honest attempt. Like the techinical notes, I initialize a min heap in get_huffman_roots. I made a mistake in get_smallest_root since I thought using pop() on the min heap would be sufficient, but now I realize it wasn't needed. This leads to another issue in my huffman message where I call get_huffman_roots when I could just use pop(). It functions the same, but it is inefficient. My encode method is the same as the technical notes. Even though my code could be more sophisticated, it was an honest attempt I'm proud of.